# Explore here

Imports

In [186]:
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

First we load the data to then create and fill the database with it.

In [187]:
movies_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")


Now create and connect to the database and then use to_sql method to transform both datasets into new tables.

In [188]:
#Create the connection
connection = sqlite3.connect("../data/movies_db.db")

#Create the new tables using both dataframes
movies_data.to_sql("movies_table", connection, if_exists= "replace", index= False)
credits_data.to_sql("credits_table", connection, if_exists= "replace", index= False)

4803

Join the two tables to create a new one sing the title column of both tables

In [189]:
#Join the tables with a query that also keeps only the columns that the exercise needs
query = """
    SELECT credits_table.movie_id, 
           movies_table.title, 
           movies_table.overview, 
           movies_table.genres, 
           movies_table.keywords, 
           credits_table.cast, 
           credits_table.crew
    FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title;
"""

#Save the query result into a new dataset
all_data = pd.read_sql_query(query, connection)

all_data

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


Check for any duplicates or null values. None are duplicates or null

In [190]:
all_data.info()
all_data.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


np.int64(0)

Now we need to transform the data using the json library to meet the requirements of the exercise.


In [191]:
#Replace the genre and keyword column with the name part of the json
all_data["genres"] = all_data["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
all_data["keywords"] = all_data["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

#Select the three first names from the cast column
all_data["cast"] = all_data["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)

#Obtaining the name of the director from the crew column
all_data["crew"] = all_data["crew"].apply(lambda x: (" ".join([crew_member['name'] for crew_member in json.loads(x)if isinstance(crew_member, dict) and crew_member.get('job') == 'Director']) 
        if pd.notna(x) 
        and isinstance(x, str)  #Check for string value, this is necessary because if not an error will pop as one index is passed as an integer
        else None)) 

#Convert the overview (summary) column to a list
all_data["overview"] = all_data["overview"].apply(lambda x: [x])

all_data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


Now put all the data into the tags newly column, but previously make all the dataset into strings and then remove all the spaces between words

In [192]:
#Convert all the dataset info into a string
all_data["overview"] = all_data["overview"].apply(lambda x: [str(x)])
all_data["genres"] = all_data["genres"].apply(lambda x: [str(genre) for genre in x])
all_data["keywords"] = all_data["keywords"].apply(lambda x: [str(keyword) for keyword in x])
all_data["cast"] = all_data["cast"].apply(lambda x: [str(actor) for actor in x])
all_data["crew"] = all_data["crew"].apply(lambda x: [str(crew_member) for crew_member in x])


#Create the new tag column with the previous columns
all_data["tags"] = all_data["overview"] + all_data["genres"] + all_data["keywords"] + all_data["cast"] + all_data["crew"]
all_data["tags"] = all_data["tags"].apply(lambda x: ",".join(x).replace(",", " "))

#Dataset that is going to be used for the KNN
all_data.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)

Convert the all_data clean dataset to a csv

In [193]:
all_data.to_csv("../data/processed/clean_data.csv", index = False)

Now we vectorize the text

In [194]:
#Create the vectorizer
vectorizer = TfidfVectorizer()

#Vectorize the text in tags column
tfidf_matrix = vectorizer.fit_transform(all_data["tags"])

Create and train the model with the vectors

In [195]:
#Create the model
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")

#Fit the model with the vectors
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

With the model trained, create a function to generate predictions based on the movie input

In [197]:
#Function that chooses the movie title and pass it to the KNN model
def predict(movie_title):
    movie_index = all_data[all_data["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar = [(all_data["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar[1:]

predictions = predict("How to Train Your Dragon")

print(predictions)

[('How to Train Your Dragon 2', np.float64(0.6587910639692214)), ("Dragon Nest: Warriors' Dawn", np.float64(0.7813953645194456)), ("Pete's Dragon", np.float64(0.8207613622459307)), ('George and the Dragon', np.float64(0.8466219260933034)), ('Eragon', np.float64(0.8487679715646009))]
